# Elliptical, parabolic, and hyperbolic orbits using the Euler method

In the simple_two_body.ipynb notebook I implemented circular orbits with arbitary starting positions and confirmed that the truncation error converged linearly with timestep for the Euler method.  

In this notebook I plan to implement hyperbolic, elliptical, and parabolic orbits using the Euler method. Parabolic orbits are marginally bound, hyperbolic orbits are unbound, and elliptical orbits are bound but oblong. All orbits should have the center of mass at one focus for both stars orbit and be symmetric between the two stars. 

Elliptical orbits should be elongated away from the other star. 

Hyperbolic and parabolic orbits should open away from the other star. 

All orbits should conserve energy and angular momentum. Later this could be enforced. I will probably not do that in this notebook. 

It would be meaningless to evaluate the L0 error for a hyperbolic or parabolic orbit since there is no cycle and the L0 error is found after one or more cycles when the system returns to its starting point. However, it would be possible to evaluate it for the elliptical orbit system, and I intend to check that the convergence of the L0 error with dt is linear as it should be for the Euler method. 

Because the convergence can be tested for ellipses only and because ellipses are closed so make it clear if they are not working correctly graphically, I will begin with ellipses then proceed to parabolas and hyperbolas. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

In [2]:
def euler(h,t, x,y,z,f,debugprint):
    xstep = (f(t, x, y,z)+f(t+h,x+h,y,z))/2*h
    xnew=x+xstep
    if(debugprint):
        print(h,t,x,xstep,xnew)
    return t+h,xnew

In [3]:
import math
def InitialDataEqualMassConic(ecc,angle,initmass):
    orbitalangle=angle
    phi=np.array([orbitalangle,math.pi+orbitalangle])
    orbitalradius=100*np.ones(2) #semimajor axis
    eccentricity=ecc
    print(eccentricity)
    mass=np.ones(2)
    masses=initmass*mass #*masssun (natural units)
    return phi,orbitalangle,orbitalradius,eccentricity, masses
    

In [4]:
import random,numpy as np
initdateqellipse=InitialDataEqualMassConic(0.1,0.,1.)
print(initdateqellipse)

0.1
(array([0.        , 3.14159265]), 0.0, array([100., 100.]), 0.1, array([1., 1.]))


In [38]:
def getxyuveqconicredmass(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    print("initdat", orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    #fix x0 y0 at one star, disregard initial data, use orbital radius as separation between stars
    #this is consistant with choice in previous part
    cosphi=np.cos(phi)
    sinphi=np.sin(phi)
    coordsep=orbitalradius #/2.
    print("coordsep",coordsep)
    #x0=orbitalradius/2.*np.cos(phi)*metersperAU
    #y0=orbitalradius/2.*np.sin(phi)*metersperAU
    count=0
    for phi0 in phi:
        if phi0==0:
            print("zero")
            #x0[count]=orbitalradius[count]/2.
            #y0[count]=0
            cosphi[count]=1.0
            sinphi[count]=0.0
        if phi0==math.pi:
            cosphi[count]=-1.0
            sinphi[count]=0.0
            print("pi")
            #x0[count]=-orbitalradius[count]/2.
            #y0[count]=0
        if phi0==math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=1.0
            #x0[count]=0
            #y0[count]=orbitalradius[count]/2.
        if phi0==3*math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=-1.0
            #x0[count]=0
            #y0[count]=-orbitalradius[count]/2.
        count+=1
    x0=coordsep*cosphi
    y0=coordsep*sinphi
    #x0[1]=0.0
    #y0[1]=0.0
    z0=np.zeros(2)
    print(x0)
    print(y0)
    v=np.zeros(2)
    a=np.zeros(2)
    ax0=0.
    ay0=0.
    az0=0.
    vx0=0.
    vy0=0.
    vz0=0.
    print(eccentricity)
    if eccentricity<1 and eccentricity>0:
        #elliptical
        orbitalr=orbitalradius[0]/2. #reduced mass
        E=-Gconstant*masses[0]*masses[1]/(2.*orbitalr) #orbitalradius is correct here r1+r2.
        #This factor of half is due to the virial (VEE-REE-AHL) theorem, a stellar dynamics theorem governing the 
        #equipartion of degrees of freedom of energy in a system. Another way of writing it is T=-1/2*V. 2T-V=0
        coordsep=orbitalr*(1.+eccentricity) #starts at apastron
        starsep=coordsep #reduced mass, M is at center
        Eapastron=-Gconstant*masses[0]*masses[1]/(2.*starsep) #factor of 2 virial again
        Fapastron=Eapastron/(starsep)
        print("E,F",E,Fapastron,2*orbitalr,starsep)
        x0=coordsep*cosphi
        y0=coordsep*sinphi
        x0[0]=0.0
        y0[0]=0.0
        v=np.zeros(2)
        v[0]=0.0
        v[1]=np.sqrt(2.*(E-2*Eapastron)/masses[1]) #1/2*mv^2-GMm/starsep=E. Solve for v. Eapastron=-GMm/2starsep
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        a[0]=0.0
        a[1]=Fapastron/masses[1]
    
        ax0=-a*cosphi
        ay0=-a*sinphi
        az0=np.zeros(2)
    elif eccentricity[0]==0.0: #circular
        #start at perihelion for both (eliptical, doesn't generalize to three body)
        #actually start with circular orbit
        ux0=np.zeros(2) #*149597870700
        #centrepital force balances gravitational force
        metersperAU=1 #natural units
        #G=1
        Gconstant=1
        #Fcentripital=mass1*v**2/rphys
        #centF=accel
        #Faccel=G*m1*m2/r^2
        r0=2.*orbitalradius #Mystery factor of 2
        print("r0", r0)
        v=np.zeros(2)
        for i in np.arange(2):
            v[i]=np.sqrt(Gconstant*masses[(i+1)%2]/np.abs(r0[i]))
        print(v)
        #r0=orbitalradius #np.sqrt(x0**2+y0**2)
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        for i in np.arange(2):
            a[i]=Gconstant*masses[(i+1)%2]/(r0[i]**2)
    
        ax0=-a*cosphi
        ay0=-a*sinphi
        az0=np.zeros(2)
    
    
    return masses,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [39]:
xyuvaeqconic=getxyuveqconicredmass(initdateqellipse)
print(xyuvaeqconic)

initdat [100. 100.] [0.         3.14159265] [ 1. -1.] [0.0000000e+00 1.2246468e-16]
coordsep [100. 100.]
zero
pi
[ 100. -100.]
[0. 0.]
0.1
E,F -0.01 -0.00016528925619834707 100.0 55.00000000000001
(array([1., 1.]), array([  0., -55.]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([ 0.        , -0.12792043]), array([0., 0.]), array([-0.        , -0.00016529]), array([-0.,  0.]), array([0., 0.]))


In [43]:
def timestep(step,t,dt,mtotal, mass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vxi))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vyi))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=np.zeros(np.size(axi))
    ayii=np.zeros(np.size(ayi))
    azii=np.zeros(np.size(azi))
    
    #This is redundant in the center of mass system
    Gconstant=6.408*10**-11
    Gconstant=1
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                if j==0: #reduced mass
                    axii[0]=0.0
                    ayii[0]=0.0
                    azii[0]=0.0
                else:
                    rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                    #print(rreljk)
                    axii[j]-=Gconstant*mass[k]*(xi[j]  - xi[k])/rreljk**3
                    ayii[j]-=Gconstant*mass[k]*(yi[j]  - yi[k])/rreljk**3
                    azii[j]-=Gconstant*mass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        vxii[m] = vxi[m] + dt*axi[m]
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
        
        #print(vyii[m],vyi[m],ayi[m],dt)
        #print(yii[m],yi[m],vyi[m],dt)
    
    
    return mass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

In [44]:
dt=1 #*31556926 #seconds per year
numsteps=10000
mass0,x,y,z0,vx,vy,vz0,ax,ay,az0=xyuvaeqconic
print(x)
xcoord1=[]
xcoord2=[]
ycoord1=[]
ycoord2=[]
t=0.0
masstotal=1.
for i in np.arange(1,numsteps):
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,masstotal,mass0,x,y,z0,vx,vy,vz0,ax,ay,az0)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
    xcoord1.append(x[0])
    xcoord2.append(x[1])
    ycoord1.append(y[0])
    ycoord2.append(y[1])
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[  0. -55.]
[  0. -55.] [ 0.         -0.12792043] [-0.         -0.00016529] [ 0.         -0.12792043] [0.         0.00033058] [0. 0.]
[  0.         -55.00016529] [ 0.         -0.25584086] [0.         0.00016529] [ 0.         -0.12792043] [0.         0.00033058] [0.00000000e+00 7.68861859e-07]
[  0. -55.] [ 0.         -0.38376129] [0.         0.00049587] [ 0.         -0.12791966] [0.         0.00033057] [0.00000000e+00 1.53767242e-06]
[  0.         -54.99950413] [ 0.         -0.51168095] [0.         0.00082643] [ 0.         -0.12791812] [0.         0.00033055] [0.00000000e+00 2.30643586e-06]
[  0.        -54.9986777] [ 0.         -0.63959907] [0.         0.00115699] [ 0.         -0.12791582] [0.         0.00033054] [0.0000000e+00 3.0751517e-06]
[  0.         -54.99752072] [ 0.         -0.76751489] [0.         0.00148753] [ 0.         -0.12791274] [0.         0.00033053] [0.0000000e+00 3.8438195e-06]
[  0.         -54.99603319] [ 0.         -0.89542763] [0.         0.00181805] [ 0.      

[  0.         -48.62007059] [  0.         -24.59369786] [0.        0.0636294] [ 0.        -0.1128692] [0.         0.00030087] [0.        0.0001513]
[  0.         -48.55644119] [  0.         -24.70656706] [0.         0.06393028] [ 0.         -0.11271791] [0.         0.00030057] [0.         0.00015204]
[  0.         -48.49251092] [  0.         -24.81928496] [0.         0.06423085] [ 0.         -0.11256586] [0.         0.00030028] [0.         0.00015279]
[  0.         -48.42828006] [  0.         -24.93185083] [0.         0.06453113] [ 0.         -0.11241308] [0.         0.00029997] [0.         0.00015353]
[  0.         -48.36374894] [  0.         -25.04426391] [0.        0.0648311] [ 0.         -0.11225955] [0.         0.00029967] [0.         0.00015428]
[  0.         -48.29891784] [  0.         -25.15652345] [0.         0.06513077] [ 0.         -0.11210527] [0.         0.00029937] [0.         0.00015502]
[  0.         -48.23378707] [  0.         -25.26862872] [0.         0.06543014] [ 0.

[  0.         -39.51090142] [  0.         -36.30881082] [0.         0.09561546] [ 0.         -0.09088791] [0.         0.00025621] [0.         0.00023429]
[  0.         -39.41528597] [  0.         -36.39969874] [0.         0.09587166] [ 0.         -0.09065363] [0.         0.00025571] [0.         0.00023499]
[  0.         -39.31941431] [  0.         -36.49035236] [0.         0.09612738] [ 0.         -0.09041864] [0.         0.00025522] [0.         0.00023569]
[  0.         -39.22328693] [  0.       -36.580771] [0.        0.0963826] [ 0.         -0.09018294] [0.         0.00025472] [0.        0.0002364]
[  0.         -39.12690433] [  0.         -36.67095394] [0.         0.09663732] [ 0.         -0.08994655] [0.         0.00025423] [0.        0.0002371]
[  0.         -39.03026701] [  0.         -36.76090049] [0.         0.09689155] [ 0.         -0.08970945] [0.         0.00025373] [0.        0.0002378]
[  0.         -38.93337547] [  0.         -36.85060994] [0.         0.09714527] [ 0.    

[  0.         -22.86994212] [  0.         -46.85186538] [0.         0.12716244] [ 0.         -0.04893111] [0.         0.00016217] [0.         0.00033005]
[  0.         -22.74277968] [  0.        -46.9007965] [0.         0.12732461] [ 0.         -0.04860107] [0.         0.00016138] [0.         0.00033061]
[  0.         -22.61545507] [  0.         -46.94939757] [0.         0.12748599] [ 0.         -0.04827045] [0.         0.00016059] [0.         0.00033118]
[  0.         -22.48796908] [  0.         -46.99766802] [0.         0.12764659] [ 0.         -0.04793927] [0.        0.0001598] [0.         0.00033175]
[  0.         -22.36032249] [  0.         -47.04560729] [0.         0.12780639] [ 0.         -0.04760753] [0.         0.00015901] [0.         0.00033231]
[  0.        -22.2325161] [  0.         -47.09321482] [0.        0.1279654] [ 0.         -0.04727522] [0.         0.00015821] [0.         0.00033287]
[  0.         -22.10455071] [  0.         -47.14049004] [0.         0.12812361] [ 0.

[ 0.         -0.05288895] [  0.         -50.06777696] [0.         0.14171263] [0.         0.01272453] [0.00000000e+00 1.54929785e-06] [0.         0.00039871]
[0.         0.08882368] [  0.         -50.05505243] [0.         0.14171418] [0.         0.01312324] [0.00000000e+00 4.21394933e-07] [0.         0.00039892]
[0.         0.23053786] [  0.         -50.04192919] [0.        0.1417146] [0.         0.01352216] [ 0.00000000e+00 -7.08244055e-07] [0.         0.00039912]
[0.         0.37225246] [  0.         -50.02840703] [0.         0.14171389] [0.         0.01392128] [ 0.00000000e+00 -1.83961225e-06] [0.         0.00039932]
[0.         0.51396635] [  0.         -50.01448575] [0.         0.14171205] [0.         0.01432059] [ 0.00000000e+00 -2.97270274e-06] [0.         0.00039951]
[0.        0.6556784] [  0.         -50.00016516] [0.         0.14170908] [0.         0.01472011] [ 0.00000000e+00 -4.10750851e-06] [0.         0.00039971]
[0.         0.79738748] [  0.         -49.98544505] [0.   

...

[ 0.        54.7088694] [0.         5.70521224] [ 0.         -0.01135881] [0.         0.14202699] [ 0.         -0.00032886] [ 0.00000000e+00 -3.34338498e-05]
[ 0.         54.69751059] [0.         5.84723923] [ 0.         -0.01168767] [0.         0.14199355] [ 0.         -0.00032873] [ 0.00000000e+00 -3.42809162e-05]
[ 0.         54.68582291] [0.         5.98923278] [ 0.        -0.0120164] [0.         0.14195927] [ 0.        -0.0003286] [ 0.00000000e+00 -3.51273819e-05]
[ 0.         54.67380651] [0.         6.13119205] [ 0.       -0.012345] [0.         0.14192414] [ 0.         -0.00032846] [ 0.0000000e+00 -3.5973237e-05]
[ 0.         54.66146151] [0.        6.2731162] [ 0.         -0.01267346] [0.         0.14188817] [ 0.         -0.00032832] [ 0.00000000e+00 -3.68184712e-05]
[ 0.         54.64878806] [0.         6.41500437] [ 0.         -0.01300178] [0.         0.14185135] [ 0.         -0.00032818] [ 0.00000000e+00 -3.76630747e-05]
[ 0.         54.63578628] [0.         6.55685572] [ 0.

[ 0.         49.45246099] [ 0.         26.18641153] [ 0.         -0.05787814] [0.         0.12806549] [ 0.        -0.0002827] [ 0.         -0.00014879]
[ 0.         49.39458286] [ 0.         26.31447701] [ 0.         -0.05816083] [0.        0.1279167] [ 0.         -0.00028223] [ 0.         -0.00014945]
[ 0.         49.33642202] [ 0.         26.44239371] [ 0.         -0.05844307] [0.         0.12776725] [ 0.         -0.00028177] [ 0.         -0.00015011]
[ 0.         49.27797896] [ 0.         26.57016095] [ 0.         -0.05872483] [0.         0.12761714] [ 0.        -0.0002813] [ 0.         -0.00015077]
[ 0.         49.21925412] [ 0.         26.69777809] [ 0.         -0.05900613] [0.         0.12746637] [ 0.         -0.00028083] [ 0.         -0.00015142]
[ 0.         49.16024799] [ 0.         26.82524447] [ 0.         -0.05928696] [0.         0.12731495] [ 0.         -0.00028036] [ 0.         -0.00015207]
[ 0.         49.10096103] [ 0.         26.95255942] [ 0.         -0.05956732] [0. 

[ 0.         39.10481044] [ 0.         42.09442904] [ 0.         -0.09168993] [0.         0.10231433] [ 0.         -0.00020679] [ 0.         -0.00022154]
[ 0.         39.01312051] [ 0.         42.19674338] [ 0.         -0.09189673] [0.         0.10209279] [ 0.         -0.00020618] [ 0.         -0.00022194]
[ 0.         38.92122378] [ 0.         42.29883617] [ 0.        -0.0921029] [0.         0.10187085] [ 0.         -0.00020556] [ 0.         -0.00022233]
[ 0.         38.82912088] [ 0.         42.40070702] [ 0.         -0.09230846] [0.         0.10164852] [ 0.         -0.00020493] [ 0.         -0.00022272]
[ 0.         38.73681243] [ 0.         42.50235554] [ 0.         -0.09251339] [0.        0.1014258] [ 0.         -0.00020431] [ 0.         -0.00022311]
[ 0.         38.64429904] [ 0.         42.60378135] [ 0.        -0.0927177] [0.        0.1012027] [ 0.         -0.00020369] [ 0.         -0.00022349]
[ 0.         38.55158133] [ 0.         42.70498404] [ 0.        -0.0929214] [0.     

[ 0.         25.56394127] [ 0.         53.49018433] [ 0.         -0.11337911] [0.         0.07065971] [ 0.         -0.00012332] [ 0.         -0.00025656]
[ 0.         25.45056216] [ 0.         53.56084404] [ 0.         -0.11350243] [0.         0.07040315] [ 0.         -0.00012269] [ 0.         -0.00025671]
[ 0.         25.33705973] [ 0.         53.63124719] [ 0.         -0.11362512] [0.         0.07014644] [ 0.         -0.00012205] [ 0.         -0.00025685]
[ 0.         25.22343461] [ 0.         53.70139363] [ 0.         -0.11374716] [0.         0.06988959] [ 0.         -0.00012141] [ 0.         -0.00025699]
[ 0.         25.10968745] [ 0.         53.77128322] [ 0.         -0.11386858] [0.        0.0696326] [ 0.         -0.00012078] [ 0.         -0.00025713]
[ 0.         24.99581887] [ 0.         53.84091582] [ 0.         -0.11398935] [0.         0.06937546] [ 0.         -0.00012014] [ 0.         -0.00025727]
[ 0.         24.88182952] [ 0.         53.91029128] [ 0.         -0.11410949] 

[0.         3.25842805] [ 0.         62.09629641] [ 0.         -0.12577927] [0.         0.02257651] [ 0.00000000e+00 -1.40865214e-05] [ 0.         -0.00025838]
[0.         3.13264877] [ 0.         62.11887292] [ 0.         -0.12579336] [0.         0.02231813] [ 0.00000000e+00 -1.35525108e-05] [ 0.         -0.00025827]
[0.         3.00685541] [ 0.         62.14119105] [ 0.         -0.12580691] [0.         0.02205986] [ 0.00000000e+00 -1.30192623e-05] [ 0.         -0.00025817]
[0.        2.8810485] [ 0.         62.16325091] [ 0.         -0.12581993] [0.        0.0218017] [ 0.0000000e+00 -1.2486777e-05] [ 0.         -0.00025806]
[0.         2.75522857] [ 0.         62.18505261] [ 0.         -0.12583242] [0.         0.02154364] [ 0.00000000e+00 -1.19550561e-05] [ 0.         -0.00025795]
[0.         2.62939615] [ 0.         62.20659624] [ 0.         -0.12584437] [0.         0.02128569] [ 0.00000000e+00 -1.14241007e-05] [ 0.         -0.00025784]
[0.         2.50355178] [ 0.         62.227881

In [45]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="dt=1, Euler Method, Equal Mass System, Ellipse", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(xcoord1,ycoord1, size=1, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="Star 1")
p.circle(xcoord2,ycoord2, size=1, line_color="blue", fill_color="blue", fill_alpha=0.5,legend="Star 2")

p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

This doesn't handle the mass conversion to reduced mass system correctly yet but I want to check it in while I have an orbit. 